# 1. Segmentation

In the final step, we will perform the segmentation and wear measurement. Your task will be to receive unseen images for the model, perform segmentation on these images, and finally measure the wear thickness to plot the corresponding VB curve for each cutting edge.

For having a better results in segmentation, you will receive a weight that has been trained on a larger dataset, resulting in improved accuracy. However, before proceeding with the segmentation task, certain preprocessing steps are required to ensure optimal results.

If you have any doubts about the previous section, please go ahead and book a consultation.

In [ ]:
from keras.models import load_model
model = load_model(r"KIP.hdf5")

# 1.1. Image Alignment
Before delving into segmentation, it's essential to address potential issues stemming from image acquisition, such as image shifting. To mitigate these issues and ensure consistency across images, we will employ image alignment techniques.

By aligning the images based on a reference or source image, we can ensure that the cutting edge, or the region of interest, remains roughly in the same position across all images. This not only simplifies the segmentation process but also improves the accuracy of subsequent measurements, such as wear thickness calculation.

With the images properly aligned, we can proceed confidently to the segmentation phase, where we'll extract meaningful insights regarding wear thickness and plot the corresponding VB curve.

You can find the images and the weight here: https://bwsyncandshare.kit.edu/s/kpdaZs6EtcM5Bpn

The test dataset comprises a complete set of cutting-edge images, starting from the beginning to the end of the tool's lifespan (approximately 200μm). As the tool has four cutting edges, four images are captured after each data collection step. The collection step occurs after every 10 tests, meaning the milling process is performed 10 times then four images are taken. This sequence is then repeated.

***Please refresh the images folder to view the images in the notebook (Download the updated images from ilias)

<div style="text-align: center;">
    <img src="Images/image-14.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>
'
<div style="text-align: center;">
    <img src="Images/image-15.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Try using the OpenCV library to implement the alignment method.

In [ ]:
import cv2 
import os 
import numpy as np 

###Import folder contains images for Image Alignment
int_folder = "Input Directory"

# Get the list of image files and skip the first one
image_files = [filename for filename in os.listdir(int_folder) if filename.endswith(('.jpg', '.jpeg', '.png'))]
image_files_source = image_files[0]
image_files_test = image_files[1:]

for filename in image_files_source: 
    # Open the image
    img_path = os.path.join(int_folder, filename)
    imgRef = cv2.imread(img_path)
    imgRef_grey = cv2.cvtColor(imgRef, cv2.COLOR_BGR2GRAY) 
    sz = imgRef_grey.shape 

# Define the motion model 
warp_mode = cv2.MOTION_TRANSLATION 

# Define 2x3 matrices and initialize the matrix to identity 
warp_matrix = np.eye(2, 3, dtype=np.float32) 

# Specify the number of iterations. 
number_of_iterations = 10000; 

# Specify the threshold of the increment 
# in the correlation coefficient between two iterations 
termination_eps = 1e-10; 

# Define termination criteria 
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)  


for filename in image_files_test: 
    # Open the image
    img_path = os.path.join(int_folder, filename)
    imgTest = cv2.imread(img_path)   
    # Convert to grayscale. 
    imgTest_grey = cv2.cvtColor(imgTest, cv2.COLOR_BGR2GRAY) 
    (cc, warp_matrix) = cv2.findTransformECC (imgRef_grey,imgTest_grey,warp_matrix, warp_mode, criteria) 
    aligned_img = cv2.warpAffine(imgTest, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP) 
    # Save the aligned image
    ALIGN = f'{filename[:-4]}_aligned.jpg'
    cv2.imwrite(ALIGN, aligned_img)
    
    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(imgRef_grey, None)
     

# 1.2. ROI Extraction

In our data preprocessing step, we have already identified the regions of interest (ROI) within the original images. Now, our task is to crop these ROIs from the test dataset as well. This is crucial for reducing the size of the images, which can streamline subsequent processing steps and improve computational efficiency.

By extracting only the relevant regions from the original images, we focus our computational resources on the areas that are most important for our analysis. This not only speeds up processing but also helps in maintaining the integrity of the data by minimizing irrelevant information.

* TASK: Crop the Region of Interest (ROI) area, as performed in the training step

In [ ]:
# Input and output directories
int_folder = "Input Directory"
out_folder = "Output Directory"

for filename in os.listdir(int_folder): 
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Open the image
        img_path = os.path.join(int_folder, filename)
        imgTest = cv2.imread(img_path)  
        imgheight = imgTest.shape[0]
        imgwidth = imgTest.shape[1]
        cropped_image = imgTest[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700] # Slicing to crop the image

        # Output filename with "_cropped" added
        output_filename = os.path.splitext(filename)[0] + "_cropped.jpg"

        # Save the cropped image to the output directory
        output_path = os.path.join(out_folder, output_filename)
        cv2.imwrite(output_path, cropped_image)
print("Cropping completed.")

# 1.3. Image resizing

In order to utilize the provided segmentation model effectively, you need to resize the images to a specific size, in this case, 512 x 512 pixels. Resizing images to a consistent dimension ensures compatibility with the model's input requirements and helps maintain the aspect ratio of the original images.

* TASK: Implement the code to resize the images to the desired dimensions

<div style="text-align: center;">
    <img src="Images/image-17.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Resize the images to match the dimensions of the training dataset

In [ ]:
# Input and output directories
int_folder = "Input Directory"
out_folder = "Output Directory"

for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add other image formats if needed
        # Open the image
        img_path = os.path.join(input_folder, filename)
        im = Image.open(img_path)

        # Resize the image to (512, 512)
        im_resized = im.resize((512, 512))

        # Save the resized image in TIFF format to the output folder
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '.jpg')
        im_resized.save(output_path, 'JPEG')

print("Resizing completed.")

# 1.4. Segmenattion Model Implemanation

Now that we have completed all the preprocessing steps, our images are prepared for segmentation. By utilizing the provided weight, you can generate the segmentation results for these preprocessed images.

ATTENTION: YOU NEED TO RESIZE THE SEGMENTATION RESULT TO THE ORIGINAL SIZE TO KEEP THE PIXEL RATIO

* TASK: Implement the segmentation code

In [ ]:
# Input and output directories
int_folder = "Input Directory"
out_folder = "Output Directory"

for filename in os.listdir(input_folder):
    if filename.endswith('.jpg'):  # Assuming you want to process TIFF images
        # Get the input image path
        img_path = os.path.join(input_folder, filename)

        # Read and normalize the image
        test_img_other = cv2.imread(img_path, 0)
        test_img_other_norm = np.expand_dims(normalize(np.array(test_img_other), axis=1),2)


        test_img_other_norm=test_img_other_norm[:,:,0][:,:,None]
        test_img_other_input=np.expand_dims(test_img_other_norm, 0)

        # Predict and threshold for values above 0.2 probability
        prediction_other = (model.predict(test_img_other_input)[0, :, :, 0] > 0.2).astype(np.uint8)

        # Resize the prediction back to the original cropped size
        prediction_other_resized = cv2.resize(prediction_other, (cropped_image.shape[1], cropped_image.shape[0]))

        # Save the prediction as an image
        output_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_prediction.png')
        Image.fromarray(prediction_other * 255).save(output_path)  # Save as PNG

print("Prediction completed.")

Here is an example of the segmentation results obtained after applying the segmentation model.

<div style="text-align: center;">
    <img src="Images/image-19.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

# 2. Wear Measurement
In this step, we are going to find out the thickness of the wear area in each image, so we can see a trend of how the wear is changing by continuing the milling process. Of course, the segmentation results are not 100% perfect, but as we just need to focus on the maximum thickness in each image, we are on the safe side. At the end of this step, there will be a value for the maximum thickness for each image. The pixel ratio that we have in the image is 1.725 μm per pixel. In this section, you have complete freedom to implement your own solution, however I will provide some hints to get you started. Let's dive into the last step :)

# 2.1. label the Target Cluster

If you check the results, you will probably see some misdetections where the model labeled areas as wear, which are not. To eliminate these artifacts and focus on the target wear area, you can implement a clustering method to identify the target area. Since the images are already aligned and the cutting edge coordinates are similar, you can also define a bounding box to locate the target cluster more accurately.

<div style="text-align: center;">
    <img src="Images/image-20.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Try to locate the right segmentation area. For this purpose, you can use findContours from OpenCV, but you can also implement your own solution

In [ ]:
def find_contour_indices(contours):
    valid_indices = []
    for i in range(len(contours)):
        if len(contours[i]) > 50:
            for j in range(len(contours[i])):
                if 450 < contours[i][j][0][0] + contours[i][j][0][1] < 550:
                    valid_indices.append(i)
                    break  # Exit the inner loop once condition is met for this contour
    return valid_indices

# 2.2. VB Measuring

In this step, we will measure the maximum thickness of the wear. You can implement a combination of functions from OpenCV, such as findContours and Canny edge detection, to find the thickness. It's important to calculate the orthogonal distance (line 1 in Max VB) each time, not the horizontal distance (line 2 in Max VB).

<div style="text-align: center;">
    <img src="Images/image-21.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: Try to measure the maximum wear thickness 

In [ ]:
directory = "Input Directory"
mg_name = []
VBmax = []

for filename in os.listdir(directory):
    img_name.append(filename) 
    img = cv2.imread(os.path.join(directory,filename), cv2.IMREAD_GRAYSCALE)
    ret,thresh = cv2.threshold(img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    result = find_contour_indices(contours)
    print(result)
    min_max_list = []
    
    for i in result:
        CNT = contours[i]
        #print(CNT)
        M = cv2.moments(CNT)
        print( M )
        RECT = cv2.minAreaRect(CNT)
        BOX = cv2.boxPoints(RECT)
        BOX = np.int0(BOX)
        FIT = cv2.drawContours(np.copy(img),[BOX],0,(255,255,255),1)
        output_path_fit = os.path.join(directory, f"{filename[:-4]}_%i_fit.png"%i)
        cv2.imwrite(output_path_fit, FIT)
        coordinates = BOX.reshape((-1, 1, 2))

        # Create a mask image with the same dimensions as the original image
        mask = np.zeros_like(img)

        # Draw a filled polygon (ROI) on the mask using the defined coordinates
        mask = cv2.fillPoly(mask, [coordinates], (255, 255, 255))

        # Perform a bitwise AND operation between the original image and the mask to extract the ROI
        roi = cv2.bitwise_and(img, img, mask=mask)

        edges = cv2.Canny(roi,180,500)
        output_path_canny = os.path.join(directory, f"{filename[:-4]}_%i_fit_canny.png"%i)
        cv2.imwrite(output_path_canny, edges) 
        White_Pixels = np.argwhere(edges == 255)
        White_Pixels_tolist = White_Pixels.tolist()
        print(White_Pixels_tolist)
        
        for y, x in White_Pixels_tolist:
            found = False
            for entry in min_max_list:
                if entry[0] == y:
                    # Update min and max x values if necessary
                    entry[1] = min(entry[1], x)  # min_x
                    entry[2] = max(entry[2], x)  # max_x
                    found = True
                    break

            if not found:
                # If y is not already in the list, append a new entry
                min_max_list.append([y, x, x, 0])  # [y, min_x, max_x, difference]

        # Calculate the difference and update the min_max_list
        for entry in min_max_list:
            entry[3] = round(np.cross(BOX[1]-BOX[0],np.array([entry[2],entry[0]])-BOX[0])/np.linalg.norm(BOX[1]-BOX[0]),2)

        # Print the result
        for entry in min_max_list:
            y, min_x, max_x, difference = entry
            #print(f"For y = {y}, min_x = {min_x}, max_x = {max_x}, difference = {difference}")
            
        Dis = [sublist[3] for sublist in min_max_list]
        VBmax.append(round(float(max(Dis))*1.725,2))

# 2.3. VB Curve

After extracting the maximum thickness of the wear in each image, as already mentioned, the images are for 4 cutting edges, each after 10 cuts. You can plot the VB curve for each cutting edge. 

<div style="text-align: center;">
    <img src="Images/image-22.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK: plot the VB curve

In [ ]:
img_name_new = []
VBmax_new = []
for i in range(0, len(img_name), 4):
    img_name_new.append(img_name[i:i+4])
    VBmax_new.append(VBmax[i:i+4])

WB = Workbook() 
Sheet1 = WB.add_sheet ('Pixel') 
style = xlwt.XFStyle ()   # create a style Object initialization pattern  
Al = xlwt.Alignment ()  
Al.horz = 0x02       # arranged horizontally centered  
style.alignment = Al  


Sheet1.write (0,0,'Tooth',style) 
Sheet1.write (1,0,'1st Tooth',style) 
Sheet1.write (2,0,'2nd Tooth',style) 
Sheet1.write (3,0,'3rd Tooth',style) 
Sheet1.write (4,0,'4th Tooth',style) 


for i in range(len(VBmax_new)): 

    Sheet1.write (0,i+1,str(i+1) +'_Track',style) 
    Sheet1.write (1,i+1,float(VBmax_new[i][0]),style)
    Sheet1.write (2,i+1,float(VBmax_new[i][1]),style)
    Sheet1.write (3,i+1,float(VBmax_new[i][2]),style)
    Sheet1.write (4,i+1,float(VBmax_new[i][3]),style) 

WB.save (r"Output Directory") 

# Extracting the tooth numbers
tooth_numbers = [f'{i}th Tooth' for i in range(1, len(VBmax_new) + 1)]

# Creating x-axis labels for all 5 cuts
x_labels = [f'{i}' for i in range(1, 970,5)]

# Transposing VBmax_new for easy plotting
vb_values_transposed = list(map(list, zip(*VBmax_new)))

# Polynomial degree
deg = 3  # You can adjust the degree as needed

# Define marker colors
marker_colors = ['red', 'green', 'blue', 'orange']

# Plotting all 5 cuts for each tooth
plt.figure(figsize=(12, 8))
for i, (vb_values, color) in enumerate(zip(vb_values_transposed, marker_colors)):
    # Plot only the markers (dots) for data points
    plt.plot(x_labels, vb_values, marker='o', linestyle='', label=f'{tooth_numbers[i]} - Data', color=color)
    # Fit a polynomial of specified degree to the data
    coefficients = np.polyfit(range(len(vb_values)), vb_values, deg)
    polynomial = np.poly1d(coefficients)
    # Plot the best-fit polynomial line
    #plt.plot(x_labels, polynomial(range(len(vb_values))), marker='', linestyle='--', label=f'{tooth_numbers[i]} - Best Fit', color=color)

plt.xlabel('Cut', fontsize=18)
plt.ylabel('VB (μm)', fontsize=18)
plt.title('VB Values for all 685 Cuts')
plt.grid(True)
plt.legend()
plt.tick_params(axis='y', which='major', labelsize=15)
plt.tick_params(axis='x', which='major', labelsize=15)

# Reduce the number of x-axis labels by selecting a subset
plt.xticks(ticks=range(0, len(x_labels), 10), labels=[x_labels[i] for i in range(0, len(x_labels), 10)], rotation=45)

plt.tight_layout()
# Save the plot as an image (optional)
plt.savefig(r"Output Directory")

# Show the plot
plt.show()

<div style="text-align: center;">
    <img src="Images/image-24.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* 2.4. Data Filtering (Extra Point)

If you have already finished up to this section, then you are officially done. Congratulations! 😉

However, if you are interested in doing an extra task, please go ahead.

After plotting the VB curve, you might notice that some results do not follow the trend and are outliers. The most obvious ones are those that are higher than the previous and the next points, which is not possible. So, try to filter the data and eliminate the artifact results.

* TASK: Implement a clustering method for data filtering and eliminate the outliers